In [1]:
import os
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

21/10/30 08:47:15 WARN Utils: Your hostname, Sumans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
21/10/30 08:47:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/30 08:47:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.3
      /_/

Using Python version 3.8.2 (default, Apr  8 2021 23:19:18)
SparkSession available as 'spark'.


#### First let's read in the processed and cleansed data

In [2]:
sc.addFile('/Users/sumangangopadhyay/complex-spark-transformations/config.py')

In [3]:
import config as cf
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType

In [4]:
clean_data_path = cf.clean_data_path()
primary_response_variables = cf.primary_response_variables().split(',')
secondary_response_variables = cf.secondary_response_variables().split(',')
primary_explanatory_variables = cf.primary_explanatory_variables().split(',')

In [5]:
df = spark.read.csv(clean_data_path, header=True)

#### Let us look for seasonality, i.e. if the incidents of violation are more in particular month(s). Note how the Issue_Month column is sorted using type casting, since this column wasn't derived as an integer before

In [6]:
df.groupBy(df.Issue_Month).count().orderBy(df.Issue_Month.cast(IntegerType())).show()

21/10/30 08:47:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------+
|Issue_Month|  count|
+-----------+-------+
|          1| 877365|
|          2| 826967|
|          3| 964737|
|          4| 888402|
|          5|1020244|
|          6| 852187|
|          7|    370|
|          8|    309|
|          9|    367|
|         10|    274|
|         11|    338|
|         12|    358|
+-----------+-------+



#### It is quite clear that the violations are most common during the first half of the year, so let's dig deeper in the first half

In [7]:
df_first6months = df.filter((df.Issue_Month.cast(IntegerType()) >= 1) \
                           | (df.Issue_Month.cast(IntegerType()) <= 6))

In [8]:
df_temp = df_first6months.groupBy(df_first6months.Issue_DayofMonth).count()

In [9]:
df_temp.createOrReplaceTempView('t_count_per_day_of_month')

In [10]:
df_temp_standardised = spark.sql("select \
                                 t1.Issue_DayofMonth, t1.count \
,round((t1.count - t2.avg_issue_count)/t2.stddev_issue_count,0) as standardised_issue_count \
,round((t1.count - t2.min_count)/(t2.max_count - t2.min_count),0) as normalised_issue_count \
from t_count_per_day_of_month t1 \
join \
(select avg(count) as avg_issue_count, stddev(count) as stddev_issue_count, min(count) as min_count \
,max(count) as max_count from t_count_per_day_of_month)t2 \
on 1=1")

In [11]:
df_temp_standardised.orderBy(df_temp_standardised.Issue_DayofMonth.cast(IntegerType())).show(40)

+----------------+------+------------------------+----------------------+
|Issue_DayofMonth| count|standardised_issue_count|normalised_issue_count|
+----------------+------+------------------------+----------------------+
|               1|192311|                     1.0|                   1.0|
|               2|195915|                     1.0|                   1.0|
|               3|225398|                     2.0|                   1.0|
|               4|179977|                     0.0|                   1.0|
|               5|162463|                     0.0|                   1.0|
|               6|211419|                     1.0|                   1.0|
|               7|179266|                     0.0|                   1.0|
|               8|187606|                     0.0|                   1.0|
|               9|170336|                     0.0|                   1.0|
|              10|182476|                     0.0|                   1.0|
|              11|166438|             

#### From the Standardised scores of Days of Months above, it seems that the violations are more common in the start and end of the months

In [12]:
df_temp = df_first6months.groupBy(df_first6months.Issue_DayofWeek).count()

In [13]:
df_temp.createOrReplaceTempView('t_count_per_day_of_week')

In [14]:
df_temp.columns

['Issue_DayofWeek', 'count']

In [15]:
df_temp_standardised = spark.sql("select \
                                 'day_'||cast(t1.Issue_DayofWeek as string) as Issue_DayofWeek, t1.count \
,round((t1.count - t2.avg_issue_count)/t2.stddev_issue_count,0) as standardised_issue_count \
,round((t1.count - t2.min_count)/(t2.max_count - t2.min_count),0) as normalised_issue_count \
from t_count_per_day_of_week t1 \
join \
(select avg(count) as avg_issue_count, stddev(count) as stddev_issue_count, min(count) as min_count \
,max(count) as max_count from t_count_per_day_of_week)t2 \
on 1=1")

In [16]:
df_temp_standardised.orderBy(df_temp_standardised.Issue_DayofWeek.cast(IntegerType())).show(40)

+---------------+------+------------------------+----------------------+
|Issue_DayofWeek| count|standardised_issue_count|normalised_issue_count|
+---------------+------+------------------------+----------------------+
|          day_1|218238|                    -2.0|                   0.0|
|          day_7|560356|                    -1.0|                   0.0|
|          day_2|869368|                     0.0|                   1.0|
|          day_4|927707|                     1.0|                   1.0|
|          day_3|961584|                     1.0|                   1.0|
|          day_6|932620|                     1.0|                   1.0|
|          day_5|962045|                     1.0|                   1.0|
+---------------+------+------------------------+----------------------+



#### Quite expectedly, the violations are more during the weekdays

#### Let us now switch our attention towards a more fine-grained view to see if the violations of specific types (Laws) are being committed by specific type of vehicles (based on their registration states) at specific places (counties where these violations occured)

In [17]:
df_temp = df_first6months.groupBy(df_first6months.Issue_DayofWeek,\
                                 df_first6months.Violation_County,\
                                 df_first6months.Law_Section,\
                                 df_first6months.Registration_State)\
.count()

#### To observe the distribution of violations per Law Section, County and Registration, let's pivot the count of violations by days of week. Also we are only keeping those combinations for which violations exist throughout the week

In [18]:
df_pivot = df_temp.groupBy('Law_Section','Violation_County','Registration_State').pivot('Issue_DayofWeek')\
.agg(func.sum('count').alias('count'))
df_pivot = df_pivot.dropna(how='any')
df_pivot.show(10)

+-----------+----------------+------------------+---+----+----+----+----+----+----+
|Law_Section|Violation_County|Registration_State|  1|   2|   3|   4|   5|   6|   7|
+-----------+----------------+------------------+---+----+----+----+----+----+----+
|        408|              BX|                CT|621|1767|1804|1533|1839|1889|1801|
|        408|               Q|                ND|  1|   2|  11|   5|   7|   8|   5|
|       1111|              BX|                NC| 26|  19|  24|  26|  25|  34|  25|
|       1111|              BX|                VA| 26|  33|  37|  35|  37|  38|  37|
|        408|               R|                NJ|141| 529| 516| 616| 549| 546| 307|
|        408|              BX|                IA|  3|  23|  30|  20|  22|  26|  18|
|       1111|              BX|                CA|  4|   5|   8|   2|   8|   5|   5|
|        408|               K|                WA| 34| 232| 265| 169| 220| 224| 141|
|        408|               K|                NV|  9|  27|  36|  22|  30|  3

In [19]:
df_temp.createOrReplaceTempView('max_violation_for_combination')

In [20]:
spark.sql("select t1.* \
from \
(select Law_Section as t1_Law_Section, Violation_County as t1_Violation_County\
, Registration_State as t1_Registration_State, max(count) as t1_max_count \
from max_violation_for_combination group by Law_Section, Violation_County, Registration_State)t1 \
join \
(select Law_Section, Violation_County, Registration_State, count from max_violation_for_combination)t2 \
on t2.Law_Section = t1.t1_Law_Section \
and t2.Violation_County = t1.t1_Violation_County \
and t2.Registration_State = t1.t1_Registration_State \
and t2.count = t1.t1_max_count").orderBy('t1_max_count',ascending=False).show(10)

+--------------+-------------------+---------------------+------------+
|t1_Law_Section|t1_Violation_County|t1_Registration_State|t1_max_count|
+--------------+-------------------+---------------------+------------+
|           408|                 NY|                   NY|      233692|
|           408|                  K|                   NY|      164702|
|           408|                  Q|                   NY|      139776|
|           408|                 BX|                   NY|       78039|
|          1180|                 BK|                   NY|       54279|
|           408|                 NY|                   NJ|       49157|
|          1180|                 QN|                   NY|       45226|
|          1180|                 BX|                   NY|       15586|
|          1111|                 QN|                   NY|       12380|
|           408|                  K|                   NJ|       12237|
+--------------+-------------------+---------------------+------

#### The above result shows that the Law 408 was the most violated in the NY county and by the vehicles registered in the state NY. This is quite expected since the entire data set is about the New York State. Let us see which combination comes on top if we take out NY from the data

In [21]:
df_temp = df_first6months.groupBy(df_first6months.Issue_DayofWeek,\
                                 df_first6months.Violation_County,\
                                 df_first6months.Law_Section,\
                                 df_first6months.Registration_State)\
.count()

In [22]:
df_temp = df_temp.filter((df_temp.Registration_State != 'NY')&(df_temp.Violation_County != 'NY'))

In [23]:
df_temp.createOrReplaceTempView('max_violation_for_combination')

In [24]:
spark.sql("select t1.* \
from \
(select Law_Section as t1_Law_Section, Violation_County as t1_Violation_County\
, Registration_State as t1_Registration_State, max(count) as t1_max_count \
from max_violation_for_combination group by Law_Section, Violation_County, Registration_State)t1 \
join \
(select Law_Section, Violation_County, Registration_State, count from max_violation_for_combination)t2 \
on t2.Law_Section = t1.t1_Law_Section \
and t2.Violation_County = t1.t1_Violation_County \
and t2.Registration_State = t1.t1_Registration_State \
and t2.count = t1.t1_max_count").orderBy('t1_max_count',ascending=False).show(10)

+--------------+-------------------+---------------------+------------+
|t1_Law_Section|t1_Violation_County|t1_Registration_State|t1_max_count|
+--------------+-------------------+---------------------+------------+
|           408|                  K|                   NJ|       12237|
|           408|                  K|                   PA|        6896|
|           408|                  Q|                   NJ|        6676|
|           408|                 BX|                   NJ|        6477|
|          1180|                 BK|                   NJ|        4153|
|           408|                  K|                   FL|        3482|
|           408|                 BX|                   PA|        3418|
|           408|                  Q|                   PA|        3254|
|           408|                  Q|                   FL|        2126|
|           408|                  K|                   CT|        2122|
+--------------+-------------------+---------------------+------

#### We can see from above that once again the Law 408 was the most violated but in the K county and by the vehicles registered in the state NJ. 

#### Since the number of rows at this level will be very high for visual inspection, we reduce the group to one attribute only

In [53]:
df_temp.columns

['Issue_DayofWeek',
 'Violation_County',
 'Law_Section',
 'Registration_State',
 'count']

In [26]:
df_temp = df_temp.withColumn('Issue_DoW',func.concat_ws('_',func.lit('day'),'Issue_DayofWeek'))

In [29]:
df_temp = df_temp.drop('Issue_DayofWeek').withColumnRenamed('Issue_DoW','Issue_DayofWeek')

In [30]:
df_temp.show(100)

+----------------+-----------+------------------+-----+---------------+
|Violation_County|Law_Section|Registration_State|count|Issue_DayofWeek|
+----------------+-----------+------------------+-----+---------------+
|              BX|        408|                MD|  475|          day_3|
|               K|        408|                CA|  526|          day_4|
|              QN|       1180|                FL|  575|          day_5|
|              BK|       1111|                MA|   61|          day_5|
|               R|        408|                VA|   34|          day_3|
|              BK|       1180|                SC|  197|          day_5|
|               R|        408|                FL|   39|          day_7|
|               K|        408|                DC|  102|          day_2|
|               K|        408|                TN|  333|          day_5|
|              BX|        408|                DE|  209|          day_5|
|              QN|       1111|                SC|   31|         

In [31]:
df_pivot = df_temp.groupBy('Law_Section').pivot('Issue_DayofWeek').sum('count')
df_pivot = df_pivot.dropna(how='any')
df_pivot.show(10)

+-----------+-----+-----+-----+-----+-----+-----+-----+
|Law_Section|day_1|day_2|day_3|day_4|day_5|day_6|day_7|
+-----------+-----+-----+-----+-----+-----+-----+-----+
|       1111| 6913| 6157| 5952| 6221| 6556| 7419| 6419|
|        408|23178|79506|87375|71419|86777|84599|63048|
+-----------+-----+-----+-----+-----+-----+-----+-----+



#### The above cell shows that the Law 408 is the most violated throughout the week

In [32]:
df_pivot = df_temp.groupBy('Violation_County').pivot('Issue_DayofWeek').sum('count')
df_pivot = df_pivot.dropna(how='any')
df_pivot.show(100)

+----------------+-----+-----+-----+-----+-----+-----+-----+
|Violation_County|day_1|day_2|day_3|day_4|day_5|day_6|day_7|
+----------------+-----+-----+-----+-----+-----+-----+-----+
|               K|10093|37675|42826|31371|41854|39254|26996|
|              MN|  682| 1514| 1452| 1500| 1513| 1668|  726|
|               Q| 5723|22064|23171|22225|23756|23558|18881|
|              BX| 7775|21666|23589|20154|23301|24396|17216|
|              QN| 2284| 5425| 5457| 6482| 6085| 6886| 2058|
|              BK| 2693|11709|12077|13475|12723|12990| 2335|
|               R|  336| 1086| 1046| 1158| 1054| 1073|  714|
|              ST|  505| 1548| 1544| 1770| 1769| 1907|  541|
+----------------+-----+-----+-----+-----+-----+-----+-----+



#### The above result shows that the K county has the most number of violations throughout the week

In [48]:
colmToOrderBy = df_temp.select(func.collect_set('Issue_DayofWeek')\
                               .alias('Issue_DayofWeek')).first()['Issue_DayofWeek']

In [49]:
df_pivot = df_temp.groupBy('Registration_State').pivot('Issue_DayofWeek').agg(func.sum('count'))
df_pivot = df_pivot.dropna(how='any')
df_pivot.orderBy(colmToOrderBy, ascending=False).show(10)

+------------------+-----+-----+-----+-----+-----+-----+-----+
|Registration_State|day_1|day_2|day_3|day_4|day_5|day_6|day_7|
+------------------+-----+-----+-----+-----+-----+-----+-----+
|                NJ| 8684|30896|34238|32486|35140|34691|20063|
|                PA| 4764|16204|17790|15068|17805|17616|11107|
|                FL| 2380| 9027| 9537| 8634| 9549| 9636| 5503|
|                CT| 2398| 6776| 7072| 5917| 7186| 7205| 5676|
|                VA| 1251| 4310| 4320| 3683| 4344| 4521| 2870|
|                MA| 1369| 3882| 3824| 3077| 3731| 4119| 3021|
|                NC|  955| 3457| 3551| 2947| 3630| 3533| 2231|
|                MD| 1091| 3359| 3393| 2908| 3439| 3624| 2762|
|                GA|  617| 2389| 2482| 2145| 2606| 2589| 1392|
|                IL|  415| 1753| 2129| 1762| 1904| 1942| 1210|
+------------------+-----+-----+-----+-----+-----+-----+-----+
only showing top 10 rows



#### This shows that the NJ registered vehicles are the most common offenders throughout the week

### Overall conclusion based on 2017 data: 
#### The Law 408 is the most violated by vehicles registered in NY and NJ. 
#### These violations are most common in K and NY counties. 
#### The violations are most common during the weekdays
#### The violations mostly occur in the beginning or ending of a month
#### Violations are more during the first half of the year